training on pseudo labelled data and original training data at the same time seems to have increased losses. what ill do here is load models trained only on training data (how my current LB was produced) and then finetune on pseudo data. this is more of a trial to see if we can lower loss.

if this doesnt work (doesnt lower loss) we can pick all the lowest loss models and just average ensemble those to see if that improves our score.

we should try different learning rates.

w/ and w/o data aug.

and different numbers of layers.


ok F all that. the mission of this nb is to pseudo label finetune xceptionet 299x299 without retraining on all the old trainign data.

In [6]:
import os, glob, bcolz, gc

import numpy as np
import pandas as pd

from tqdm import tqdm
from scipy import ndimage, misc
from scipy.stats import rankdata

from sklearn.externals import joblib
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score, log_loss, accuracy_score
from sklearn.preprocessing import MinMaxScaler

import keras
from keras import backend as K
from keras import optimizers
from keras.models import Model, load_model
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

from keras.applications.xception import preprocess_input as preprocess_input_xcep

from tensorflow.python.client import device_lib
device_lib.list_local_devices(), 'keras version: {}'.format(keras.__version__)

([name: "/cpu:0"
  device_type: "CPU"
  memory_limit: 268435456
  locality {
  }
  incarnation: 13929960804052114045, name: "/gpu:0"
  device_type: "GPU"
  memory_limit: 308019200
  locality {
    bus_id: 1
  }
  incarnation: 10139931400912383247
  physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:04:00.0"],
 'keras version: 2.0.6')

In [2]:
path = os.path.join('/scratch', 'yns207', 'data_invasive')
train_set = pd.read_csv(os.path.join(path, 'train_labels.csv'))
# updated the pseudo labels to be the latest submission which scored 0.99513 LB
test_set = pd.read_csv(os.path.join(path,'results','subm_aug14_1.gz'), compression='gzip')

def delete_model(model, clear_session=True):
    '''removes model!
    '''
    del model
    gc.collect()
    if clear_session: K.clear_session()

def read_img(img_path, img_shape):
    img = misc.imread(img_path)
    img = misc.imresize(img, img_shape)
    return img

def read_imgs(img_height, img_width):
    train_img, test_img = [],[]
    for img_path in tqdm(train_set['name'].iloc[:]):
        train_img.append(read_img(os.path.join(path, 'train', str(img_path)+'.jpg'), (img_height, img_width)))

    for img_path in tqdm(test_set['name'].iloc[:]):
        test_img.append(read_img(os.path.join(path, 'test', str(img_path)+'.jpg'), (img_height, img_width)))
    return np.array(train_img), np.array(test_img)

In [3]:
train_img, test_img = read_imgs(299,299)

100%|██████████| 1531/1531 [00:51<00:00, 25.41it/s]


In [4]:
x_tr = preprocess_input_xcep(train_img.astype(np.float32))
y_tr = train_set['invasive'].values

x_pseudo = preprocess_input_xcep(test_img.astype(np.float32))
y_pseudo = test_set['invasive'].values

x_train = np.concatenate([x_tr, x_pseudo])
y_train = np.concatenate([y_tr, y_pseudo])

In [ ]:
model_files = [
'invasive_xcep_aug11_kfolds_299x299_0.model',
'invasive_xcep_aug11_kfolds_299x299_1.model',
'invasive_xcep_aug11_kfolds_299x299_2.model',
'invasive_xcep_aug11_kfolds_299x299_3.model',
'invasive_xcep_aug11_kfolds_299x299_4.model'
]

for i,model_file_name in enumerate(model_files):
    model = load_model(os.path.join(path,model_file_name))
    model_save = os.path.join(path,'{}_{}.model'.format('invasive_xcep_aug14_pseudo_ft_299x299',i))
    
    print('original metrics before training:  roc_auc')
    print('{}'.format(roc_auc_score(y_tr, model.predict(x_tr).flatten())))
    
    model_checkpoint = ModelCheckpoint(model_save, 
                                    monitor='val_loss', 
                                    save_best_only=True)
                
    reduce_lr = ReduceLROnPlateau(monitor='loss',
                          patience=7,
                          verbose=1,
                          factor=0.1,
                          cooldown=10,
                          min_lr=0.00001)
    
    model.fit(x=x_pseudo, 
              y=y_pseudo, 
              validation_data=(x_tr,y_tr), 
              batch_size=16, 
              epochs=20,
             callbacks=[model_checkpoint, reduce_lr])
    
    r_roc_auc_score = roc_auc_score(y_tr, model.predict(x_tr).flatten())
    
    print('validation on origianl training data: roc_auc')
    print('{}'.format(r_roc_auc_score))
    
    #free memory
    delete_model(model)

original metrics before training:  roc_auc
0.9987957170905437
Train on 1531 samples, validate on 2295 samples
Epoch 1/20
1531/1531 [==============================] - 161s - loss: 0.2228 - val_loss: 0.0436